In [1]:
import numpy as np
import pandas as pd 
import lightgbm as lgb
import gc
from fastai.tabular import *
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import KFold, StratifiedKFold
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import datetime
from tqdm import tqdm_notebook

In [2]:
def rmse(y_true, y_pred):
    return np.sqrt(mse(y_true, y_pred))

In [3]:
DATA_PATH = 'input/'
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
sample_sub = pd.read_csv('SampleSubmission (1).csv')

### Converting to datetime to use date features.

In [4]:
train['Date'] = pd.to_datetime(train['Date'], format='%Y-%m-%d')
test['Date'] = pd.to_datetime(test['Date'], format='%Y-%m-%d')

In [5]:
ID_COL, TARGET_COL = 'Place_ID X Date', 'target'
df = pd.concat([train, test]).reset_index(drop=True)
features = [c for c in df.columns if c not in ['Date', 'target_count', 'target_min', 'Place_ID X Date', 'target_variance', 'Place_ID', 'target_max', 'target']]


## Feature Engineering

### Frequency Encoding

We cannot use the PlaceID directly since its different for both train and test data, but we can use the the number of times each place appears in the dataframe is not same for every place.

In [6]:
df['placeID_freq'] = df['Place_ID'].map(df['Place_ID'].value_counts())

### Time Series Features

### A. Using Previous and Future Target Values.

Pollution levels yesterday could tell about the pollution levels today, similary pollution levels tomorrow, could also tell about pollution levels today. We add this information to our dataset, using shift and expanding functions

In [7]:
for i in range(1, 20):
    df[f'prev_target_{i}'] = df.sort_values(by='Date')[TARGET_COL].fillna(method='ffill').shift(i).sort_index()
    df[f'next_target_{i}'] = df.sort_values(by='Date')[TARGET_COL].fillna(method='bfill').shift(-i).sort_index()

In [8]:
for i in tqdm_notebook(range(1, 15)):
    df[f'magic_{i}'] = df.sort_values(by='Date')[TARGET_COL].shift(i).expanding().mean().fillna(method='ffill').sort_index()
    df[f'magic2_{i}'] = df.sort_values(by='Date')[TARGET_COL].shift(-i).expanding().mean().fillna(method='bfill').sort_index()

C:\Users\etord\AppData\Local\Temp\ipykernel_11484\507106514.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(1, 15)):


  0%|          | 0/14 [00:00<?, ?it/s]

### B. Cyclic Components

Date has cyclic components, for example monday repeats after every 7 days, adding this as a feature can help capture more information from the dataset.

To read more visit:

https://docs.fast.ai/tabular.transform.html#add_cyclic_datepart

In [9]:
for i in tqdm_notebook(range(1, 45)):
    c = 'Date' + str(i+1)
    df[c] = df['Date']  + datetime.timedelta(days=i)
    # _ = add_cyclic_datepart(df, c, prefix = c)

C:\Users\etord\AppData\Local\Temp\ipykernel_11484\502876696.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(1, 45)):


  0%|          | 0/44 [00:00<?, ?it/s]

C:\Users\etord\AppData\Local\Temp\ipykernel_11484\502876696.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[c] = df['Date']  + datetime.timedelta(days=i)
C:\Users\etord\AppData\Local\Temp\ipykernel_11484\502876696.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[c] = df['Date']  + datetime.timedelta(days=i)
C:\Users\etord\AppData\Local\Temp\ipykernel_11484\502876696.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance. 

### C. Sensor Readings of previous and future days.

Similar to the pollution readings of previous and future days, we can utilize the sensor readings of previous and future days too, to help predict the pollution for today, also the difference between the sensor readings today and sensor readings from the past and future can help. I added 10 days worth of this information to the dataset, you are free to experiment with more or lesser number of days

In [10]:

    
for i in tqdm_notebook(range(1, 11)):
    tmp = df.sort_values(by='Date').groupby('Place_ID')[features].shift(i).sort_index()
    tmp_diff_prev = df[features] - tmp
    tmp.columns = [c + f'_prev_{i}' for c in tmp.columns]
    tmp_diff_prev.columns = [c + f'_prev_diff_{i}' for c in tmp_diff_prev.columns]
    df = pd.concat([df, tmp, tmp_diff_prev], axis=1)
    
    tmp = df.sort_values(by='Date').groupby('Place_ID')[features].shift(-i).sort_index()
    tmp_diff_next = df[features] - tmp
    tmp.columns = [c + f'_next_{i}' for c in tmp.columns]
    tmp_diff_next.columns = [c + f'_next_diff_{i}' for c in tmp_diff_next.columns]
    df = pd.concat([df, tmp, tmp_diff_next], axis=1)

C:\Users\etord\AppData\Local\Temp\ipykernel_11484\2402953835.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(1, 11)):


  0%|          | 0/10 [00:00<?, ?it/s]

### D. Using simple features extracted from the date itself.
Like dayofweek, weekend or not, month, days_in_month, etc.

In [11]:
for attr in ['day', 'month', 'week', 'dayofweek', 'weekofyear', 'days_in_month', 'is_month_start', 'is_month_end', 'dayofyear']:
    df[attr] = getattr(df['Date'].dt, attr)
df['is_weekend'] = (df['dayofweek'] >= 5)*1
df['fortnight'] = df['day']%15
df['which_fortnight'] = df['day']//15

# add_cyclic_datepart(df, "Date", prefix = "Current_Date_")

C:\Users\etord\AppData\Local\Temp\ipykernel_11484\3398601916.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[attr] = getattr(df['Date'].dt, attr)
C:\Users\etord\AppData\Local\Temp\ipykernel_11484\3398601916.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[attr] = getattr(df['Date'].dt, attr)


AttributeError: 'DatetimeProperties' object has no attribute 'week'

## Modelling.

1. The best idea would be do a grouped KFold validation strategy, since train and test set had different sets of places, but I realised it later, so decided to stick with Stratified KFold.
2. Using stratification for regression can be done if you divided the target into bins, here I divided the target into 10 different quantiles.
3. 10 fold validation was performed with averaging of each prediction.

In [ ]:
features = [c for c in df.columns if c not in ['Date', 'target_count', 'target_min', 'Place_ID X Date', 'target_variance', 'Place_ID',
                                               'target_max', 'target',  'month_year_cos','month_year_sin','day_year_cos','day_year_sin']]
train = df[:train.shape[0]].reset_index(drop=True)
test = df[train.shape[0]:].reset_index(drop=True)
target = train[TARGET_COL]
len(features)

3147

In [ ]:
param = {'num_leaves': 100,
         'min_data_in_leaf': 50, 
         'objective':'regression',
         'max_depth': -1,
         'learning_rate': 0.075,
         "boosting": "gbdt",
         "feature_fraction": 0.35,
         "metric": 'auc',
         "lambda_l1": 1,
         "lambda_l2": 2,
         "random_state": 6,
         "verbosity": -1,
          'metric' : 'rmse',
          'num_iterations': 1500}

In [ ]:
target_cols = ['target']
oofs_df = pd.DataFrame()
preds_df = pd.DataFrame()
for t_col in target_cols:
    oofs_df[t_col] = np.zeros(len(train))
    preds_df[t_col] = np.zeros(len(test))

In [ ]:
max_iter = 10
folds = StratifiedKFold(n_splits=max_iter, random_state=1901)

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train.values, pd.qcut(target, 10, labels=False, duplicates='drop'))):
    print("\nfold n°{}".format(fold_))
    X_trn, X_val, X_test = train.iloc[trn_idx][features], train.iloc[val_idx][features], test[features]
    for t_col in target_cols:
        target = train[t_col]
        print(f"\n\n**** {t_col} ****\n")
        y_trn, y_val = target.iloc[trn_idx], target.iloc[val_idx]
        trn_data = lgb.Dataset(X_trn, y_trn)
        val_data = lgb.Dataset(X_val, y_val)
        
        clf = lgb.train(param, trn_data, valid_sets = [trn_data, val_data], verbose_eval=50, early_stopping_rounds = 200)

        oofs_df[t_col][val_idx] = clf.predict(X_val, num_iteration=clf.best_iteration)
        current_test_pred = clf.predict(X_test, num_iteration=clf.best_iteration)
        current_test_pred[current_test_pred < 0] = 0
        preds_df[t_col] += current_test_pred / folds.n_splits

ValueError: Setting a random_state has no effect since shuffle is False. You should leave random_state to its default (None), or set shuffle=True.

## Feature Importances

On assessing feature importances we find out that,previous and next values, are indeed important. Also placeID_freq is in the top 10 features. Clever feature selection after analyzing feature importances can further help increase the score.

In [ ]:
_ = plt.figure(figsize=(10, 10))
fi = pd.Series(index=features, data=clf.feature_importance())
_ = fi.sort_values()[-20:].plot(kind='barh')

NameError: name 'plt' is not defined

In [ ]:
rmse(target.values, oofs_df['target'].values)

27.694513458631768

In [ ]:
predictions_test = preds_df['target']
predictions_test[predictions_test < 0] = 0

In [ ]:
SUB_FILE_NAME = 'preds_lgbm_v1.csv'
sub_df = pd.DataFrame()
sub_df[ID_COL] = test[ID_COL]
sub_df[TARGET_COL] = predictions_test
sub_df.to_csv(SUB_FILE_NAME, index=False)
sub_df.head(10)

,Place_ID X Date,target
0,0OS9LVX X 2020-01-02,32.871844
1,0OS9LVX X 2020-01-03,31.722997
2,0OS9LVX X 2020-01-04,30.805605
3,0OS9LVX X 2020-01-05,34.755467
4,0OS9LVX X 2020-01-06,73.882796
5,0OS9LVX X 2020-01-07,71.834860
6,0OS9LVX X 2020-01-08,29.864601
7,0OS9LVX X 2020-01-09,28.951543
8,0OS9LVX X 2020-01-10,27.678652
9,0OS9LVX X 2020-01-11,32.890756


In [ ]:
sub_df[TARGET_COL].describe()

count    16136.000000
mean        57.959790
std         35.554207
min          3.332057
25%         31.492984
50%         48.489986
75%         75.117649
max        275.325099
Name: target, dtype: float64